In [ ]:
import os
try:
    import pycaret
    import pyspark
except:
    os.system("pip install -q pycaret")
    os.system('pip install -q pyspark')
    os.system('pip install -q graphviz')
    os.system("pip install -q pydot")
    pass

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Add Function

In [ ]:
import re

def tree_to_dot(tree_str, feature_names=None):
    dot_str = 'digraph Tree {\nnode [shape=box] ;\n'
    lines = tree_str.split('\n')
    stack = []
    node_id = 0
    for line in lines:
        if line.strip() == '':
            continue
        depth = len(re.findall('\s\s', line))
        line = line.strip()
        if 'If' in line:
            if ' <= ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' <= ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} <= {value}"] ;\n'
            elif ' > ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' > ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} > {value}"] ;\n'
            elif ' in ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' in ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} in {value}"] ;\n'
            elif ' not in ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' not in ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} not in {value}"] ;\n'
            if len(stack) > 0:
                dot_str += f'{stack[-1]} -> {node_id} [labeldistance=2.5, labelangle=45, headlabel="True"];\n'
            stack.append(node_id)
            node_id += 1
        elif 'Predict' in line:
            value = re.findall(r': ([^ ]+)', line)[0]
            dot_str += f'{node_id} [label="Predict: {value}", fillcolor="#e5813900"] ;\n'
            dot_str += f'{stack[-1]} -> {node_id} ;\n'
            node_id += 1
        elif 'Else' in line:
            stack = stack[:depth]
            if ' > ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' > ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} > {value}"] ;\n'
            elif ' <= ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' <= ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} <= {value}"] ;\n'
            elif ' not in ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' not in ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} not in {value}"] ;\n'
            elif ' in ' in line:
                feature, value = re.findall(r'\(([^)]+)\)', line)[0].split(' in ')
                feature_name = feature_names[int(feature)] if feature_names else f'X[{feature}]'
                dot_str += f'{node_id} [label="{feature_name} in {value}"] ;\n'
            dot_str += f'{stack[-1]} -> {node_id} [labeldistance=2.5, labelangle=-45, headlabel="False"];\n'
            stack.append(node_id)
            node_id += 1
    dot_str += '}'
    return dot_str

In [ ]:
from pyspark.ml.classification import DecisionTreeClassificationModel
# from pyspark.ml.tree import Node

def extract_rules(node, feature_names, rule_string=""):
    if node.numChildren() == 0:
        return rule_string + "Predict: " + str(node.prediction())

    feature = node.split().featureIndex()
    threshold = node.split().threshold()
    feature_name = feature_names[feature]
    condition = " <= " if node.split().leftSide() else " > "
    rule_string += "If (" + feature_name + condition + str(threshold) + ")\n"

    left_child_rule = extract_rules(node.leftChild(), feature_names, rule_string + "  ")
    right_child_rule = extract_rules(node.rightChild(), feature_names, rule_string + "  ")
    return left_child_rule + "\n" + right_child_rule

def decision_tree_to_dot_data(dt_model, feature_names):
    tree_model = dt_model.stages[-1]
    root_node = tree_model._call_java("rootNode")

    dot_data = "digraph Tree {\n"
    dot_data += "node [shape=box]\n"

    rules = extract_rules(root_node, feature_names)
    dot_data += rules

    dot_data += "}"
    return dot_data

In [ ]:
import pandas as pd
import scipy.stats as stats
import numpy as np

def cramers_v(df):
    result = pd.DataFrame(index=df.columns, columns=df.columns)
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2:
                contingency_table = pd.crosstab(df[col1], df[col2])
                X2 = stats.chi2_contingency(contingency_table, correction=False)[0]
                n = np.sum(contingency_table.to_numpy())
                minDim = min(contingency_table.shape)-1
                V = np.sqrt((X2/n) / minDim)
                result.loc[col1, col2] = V
    return result

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif_numeric(df):
    # calculate VIF for each predictor variable
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]

    return vif_data

def calculate_vif_categorical(df):
    # create dummy variables for categorical variables
    df_dummies = pd.get_dummies(df, drop_first=True)

    # calculate VIF for each predictor variable
    vif_data = pd.DataFrame()
    vif_data["feature"] = df_dummies.columns
    vif_data["VIF"] = [variance_inflation_factor(df_dummies.values, i) for i in range(len(df_dummies.columns))]

    return vif_data

In [ ]:
from pyspark.sql.functions import col

def selectByTypePyspark(colType, df):
    cols = [field.name for field in df.schema.fields if field.dataType == colType]
    return df.select(*cols)

In [ ]:
def deselectByTypePyspark(colType, df):
    cols = [field.name for field in df.schema.fields if field.dataType != colType]
    return df.select(*cols)

In [ ]:
import re

def parse_tree_string(tree_string):
    # Split the tree string into lines
    lines = tree_string.split('\n')
    
    # Initialize variables
    nodes = []
    stack = []
    
    # Iterate over the lines
    for line in lines:
        # Skip the first line
        if line.startswith('DecisionTreeClassificationModel'):
            continue
        
        # Find the depth of the current node
        depth = line.find('If')
        
        # Check if the line represents a decision node
        if depth >= 0:
            # Find the feature and threshold of the current node
            feature, threshold = re.findall(r'feature (\d+) <= ([\d.]+)', line)[0]
            
            # Create a new node
            node = {'feature': int(feature), 'threshold': float(threshold), 'left': None, 'right': None}
        else:
            # Find the prediction of the current node
            prediction = re.findall(r'Predict: ([\d.]+)', line)[0]
            
            # Create a new node
            node = {'prediction': float(prediction)}
            
            # Set the depth of the current node
            depth = len(stack) - 1
        
        # Add the node to the list of nodes
        nodes.append(node)
        
        # Update the stack
        while len(stack) > depth:
            stack.pop()
        
        # Update the parent node
        if len(stack) > 0:
            parent_node = nodes[stack[-1]]
            if parent_node['left'] is None:
                parent_node['left'] = len(nodes) - 1
            else:
                parent_node['right'] = len(nodes) - 1
        
        # Add the current node to the stack
        stack.append(len(nodes) - 1)
    
    # Return the root node
    return nodes[0]


In [ ]:
from graphviz import Digraph

def plot_tree(node, graph=None, parent=None, edge_label=None):
    # Create a new graph
    if graph is None:
        graph = Digraph()
    
    # Get the node ID
    node_id = str(id(node))
    
    # Add the node to the graph
    if 'prediction' in node:
        graph.node(node_id, label=str(node['prediction']))
    else:
        graph.node(node_id, label=f'X[{node["feature"]}] <= {node["threshold"]}')
    
    # Add the edge to the graph
    if parent is not None:
        graph.edge(parent, node_id, label=edge_label)
    
    # Recursively plot the child nodes
    if 'left' in node:
        plot_tree(node['left'], graph=graph, parent=node_id, edge_label='True')
    if 'right' in node:
        plot_tree(node['right'], graph=graph, parent=node_id, edge_label='False')
    
    # Return the graph
    return graph

# Input Data

In [ ]:
loan_df = pd.read_csv('/kaggle/input/rakamin-idx/loan_data_2007_2014.csv')
loan_df.head()

# Cleaning

In [ ]:
reject_stats = ['url',
 'zip_code',
 'title',
 'mths_since_last_record',
 'mths_since_last_delinq',
 'issue_d',
 'inq_last_6mths',
 'emp_title',
 'emp_length',
 'earliest_cr_line',
 'desc',
 'addr_state'
]

In [ ]:
loan_df = loan_df.drop(reject_stats,axis=1)
loan_df

In [ ]:
loan_df.term.unique()

In [ ]:
loan_df.term.replace({' 36 months':'36 months',
                      ' 60 months':'60 months'},
                     inplace=True)

In [ ]:
del_col = loan_df.isna().sum()[loan_df.isna().sum()==(loan_df.shape[0])].index
del_col

In [ ]:
loan_df = loan_df.drop(del_col,axis=1)
loan_df

In [ ]:
loan_df = loan_df.drop("Unnamed: 0",axis=1)
loan_df

In [ ]:
loan_df = loan_df.drop(['id','member_id'],axis=1)
loan_df.head()

In [ ]:
miss_col = loan_df.isna().sum()[loan_df.isna().sum()!=0]/loan_df.shape[0]*100
miss_col

In [ ]:
miss_col = miss_col[miss_col>20]
miss_col

In [ ]:
miss_col = miss_col.index.tolist()
miss_col

In [ ]:
loan_df = loan_df.drop(miss_col,axis=1)
loan_df.head()

In [ ]:
for colname in loan_df.columns:
    banyak = len(loan_df[colname].unique())
    if(banyak==1):
        print(colname,banyak)
loan_df = loan_df.drop(['policy_code','application_type'],axis=1)
loan_df

In [ ]:
loan_df.select_dtypes(exclude='object').corr()

In [ ]:
loan_df.isna().sum()[loan_df.isna().sum()!=0]

# Imputation

In [ ]:
from statsmodels.imputation.mice import MICEData

loan_df_num = loan_df.select_dtypes(exclude=['object','datetime64'])
imp = MICEData(loan_df_num)
imp.update_all(5)
loan_df_num = imp.data
loan_df[loan_df_num.columns.tolist()] = loan_df_num
loan_df.isna().sum()

In [ ]:
numeric_vif = calculate_vif_numeric(loan_df_num)
numeric_vif = numeric_vif.sort_values('VIF',ascending=False)
numeric_vif

In [ ]:
drop_vif = ['total_pymnt',
            'out_prncp',
            'funded_amnt',
            'funded_amnt_inv',
            'total_pymnt_inv',
            'loan_amnt',
            'installment',
            'int_rate',
            'open_acc'
            ]

In [ ]:
numeric_vif = calculate_vif_numeric(loan_df_num.drop(drop_vif,
                                                     axis=1))
numeric_vif = numeric_vif.sort_values('VIF',ascending=False)
numeric_vif

In [ ]:
loan_df_num = loan_df_num[numeric_vif.feature.to_list()]
loan_df_num

In [ ]:
loan_df = loan_df.drop(drop_vif,axis=1)
loan_df

In [ ]:
loan_df.last_pymnt_d = pd.to_datetime(loan_df.last_pymnt_d,format="%b-%y")
loan_df.last_credit_pull_d = pd.to_datetime(loan_df.last_credit_pull_d,format="%b-%y")
loan_df

In [ ]:
cramers_v(loan_df.select_dtypes(include='object'))

In [ ]:
objek_vif = calculate_vif_categorical(loan_df.select_dtypes(include='object').drop(['sub_grade'],axis=1))
objek_vif = objek_vif.sort_values('VIF',ascending=False)
objek_vif

In [ ]:
loan_df = loan_df.drop(['sub_grade'],axis=1)
loan_df

# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=loan_df,
              y='loan_status',
              order=loan_df['loan_status'].value_counts().index
             )

# Add annotations to display the count of each category
for p in plt.gca().patches:
    plt.gca().text(p.get_width(), 
                   p.get_y() + p.get_height() / 2.,
                   f'{int(p.get_width())}',
                   va='center',
                  )
# Show the plot
plt.show()

Dari Data ini membuktikan bahwa :
* Disini ada lebih banyak status utang, current ,fully paid, dan chargeoff
* Ada terjadinya ketidakseimbangan antar kelasnya

In [ ]:
# sns.pairplot(loan_df.select_dtypes(include=['float64']))

# Example MLspark

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql import SparkSession


# Create an instance of SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Create a larger dummy dataset with 20 rows, multiple categorical and numeric columns, and a multiclass label column
data = spark.createDataFrame([
    ("A", "X", "M", "S", "U", "K", 1.0, 2.0, 3.0, 0),
    ("B", "Y", "N", "T", "V", "L", 2.0, 3.0, 4.0, 1),
    ("C", "Z", "O", "U", "W", "M", 3.0, 4.0, 5.0, 2),
    ("A", "X", "P", "V", "X", "N", 4.0, 5.0, 6.0, 2),
    ("B", "Y", "Q", "W", "Y", "O", 5.0, 6.0, 7.0, 1),
    ("C", "Z", "R", "X", "Z", "P", 6.0, 7.0, 8.0, 2),
    ("A", "X", "S", "Y", "A", "Q", 7.0, 8.0, 9.0, 1),
    ("B" ,"Y" ,"T" ,"Z" ,"B" ,"R" ,8.0 ,9.0 ,10.0 ,2),
    ("C" ,"Z" ,"U" ,"A" ,"C" ,"S" ,9.0 ,10.0 ,11.0 ,1),
    ("A" ,"X" ,"V" ,"B" ,"D" ,"T" ,10.0 ,11.0 ,12.0 ,2),
    ("B" ,"Y" ,"W" ,"C" ,"E" ,"U" ,11.0 ,12.0 ,13.0 ,1),
    ("C" ,"Z" ,"X" ,"D" ,"F" ,"V" ,12.0 ,13.0 ,14.0 ,2),
    ("A" ,"X" ,"Y" ,"E" ,"G" ,"W" ,13.0 ,14.0 ,15.0 ,1),
    ("B","Y","Z","F","H","X",14.0,15.0,16.0,2),
    ("C","Z","A","G","I","Y",15.0,16.0,17.0,1),
    ("A","X","B","H","J","Z",16.0,17.0,18.0,2),
    ("B","Y","C","I","K","A",17.0,18.0,19.0,1),
    ("C","Z","D","J","L","B",18.0,19.0,20.0,2),
    ("A","X","E","K","M","C,",19.0,20.0,21.0,1),
    ("B",'Y','F','L','N','D',20.0,21.0,22.0,2)
], ["category1",
     'category2',
     'category3',
     'category4',
     'category5',
     'category6',
     'numeric1',
     'numeric2',
     'numeric3',
     'label'])



# Define the categorical and numeric columns
categorical_cols = ['category1',
                    'category2',
                    'category3',
                    'category4',
                    'category5',
                    'category6']
numeric_cols = ['numeric1','numeric2','numeric3']

# Create instances of StringIndexer and OneHotEncoder for each categorical column
indexers = [StringIndexer(inputCol=colu,outputCol=f"{colu}Index") for colu in categorical_cols]
encoders = [OneHotEncoder(inputCol=f"{colu}Index", outputCol=f"{colu}Vec") for colu in categorical_cols]

# Assemble the one-hot encoded and numeric columns into a single features column
assembler = VectorAssembler(inputCols=[f"{colu}Vec"
                                       for colu in categorical_cols] + numeric_cols,
                            outputCol="features")

# Create an instance of DecisionTreeClassifier for multiclass classification
dtc = DecisionTreeClassifier(labelCol="label",
                             featuresCol ="features")

# Create a pipeline with the stages
pipeline = Pipeline(stages=indexers + encoders + [assembler] + [dtc])

# Fit the pipeline to the data
model = pipeline.fit(data)

# Make predictions on the data
predictions = model.transform(data)

print(predictions.show())
# Show the predictions
predictions.select("prediction" ,"label").show()

# Create an instance of MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                              predictionCol="prediction",
                                              metricName="accuracy")

# Evaluate the model on the data
accuracy = evaluator.evaluate(predictions)

# Print the accuracy
print(f"Accuracy: {accuracy}")

# Machine Learning

In [ ]:
loan_df = loan_df.fillna(loan_df.mode().iloc[0])
loan_df.isna().sum()[loan_df.isna().sum()!=0]

In [ ]:
loan_df

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName("PandasToPySpark").getOrCreate()
spark_df = spark.createDataFrame(loan_df)
spark_df

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

cat_col = ['term',
           'grade',
           'home_ownership',
           'verification_status',
           #'loan_status',
           'pymnt_plan',
           'purpose',
           'initial_list_status']
# One-hot encode the categorical variables
indexers = [StringIndexer(inputCol=colu, outputCol=colu+"_index") for colu in cat_col]
encoders = [OneHotEncoder(inputCol=colu+"_index", outputCol=colu+"_vec") for colu in cat_col]
label_indexer = StringIndexer(inputCol="loan_status", outputCol="loan_status_index")
pipeline = Pipeline(stages=indexers + encoders + [label_indexer])
spark_df = pipeline.fit(spark_df).transform(spark_df)
spark_df.printSchema()

In [ ]:
spark_df = spark_df.drop('last_pymnt_d','last_credit_pull_d')
spark_df.printSchema()

In [ ]:
from pyspark.sql.types import StringType
spark_df_non_str = deselectByTypePyspark(StringType(),spark_df)
spark_df_non_str

In [ ]:
cc = spark_df_non_str.columns
cc.remove('loan_status_index')
cc = [c for c in cc if (c.endswith('_index')==False)]
cc

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

coba_df = spark_df_non_str

from pyspark.ml.feature import VectorAssembler
assem = VectorAssembler(inputCols=cc,
                        outputCol='feature_index',
                       )
coba_df = assem.transform(coba_df)

from pyspark.ml.feature import VectorAssembler
assem_lab = VectorAssembler(inputCols=['loan_status_index'],
                            outputCol='loan_status_vec',
                       )
coba_df = assem_lab.transform(coba_df)

print(coba_df.select('feature_index').show(5))


train_df,test_df = coba_df.randomSplit([0.8,0.2],seed=42)
dt = DecisionTreeClassifier(featuresCol="feature_index", 
                            labelCol="loan_status_index",
                            maxDepth=4,
                            maxBins=40,
                            minInstancesPerNode=2,
                           )
model = dt.fit(train_df)
predictions = model.transform(test_df)
print(predictions.select('prediction','loan_status_index').show(5))



# Create an instance of MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="loan_status_index",
                                              predictionCol="prediction",
                                              metricName="accuracy")

# Evaluate the model on the test dataset
accuracy = evaluator.evaluate(predictions)
# Print the accuracy
print(f"Accuracy: {accuracy}")

In [ ]:
import dtreeviz
import graphviz
import pydot
from pyspark.ml.classification import DecisionTreeClassificationModel
from graphviz import Source


# Extract the trained decision tree model
trained_model = model

# Extract the tree structure and rules
tree_string = trained_model.toDebugString
dot_string = tree_to_dot(tree_string)
src = Source(dot_string)
src.format='png'
src.render('decision-tree.gv', view=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure


im = img.imread('/kaggle/working/decision-tree.gv.png')
figure(figsize=(16, 8))
plt.imshow(im)
plt.show()

In [ ]:
print(dot_string)